In [1]:
import pandas as pd
import requests

gas_df = pd.DataFrame()
# Hardcoded API key that will max out at 500 calls its alr at like 70 
# so if you re-run this file and its not working the calls are prob maxed out
# and you have to make an acct for yourself
api_key = 'gxsUNM0ko2FvmZc77RZpoTsZTmONN3UP'
fuel_type = 'Unleaded'
car_type = 'used'
rows_per_page = 25

def get_gas_data(api_key, fuel_type, car_type, rows_per_page):
    global gas_df
    for page in range(0, 10):  # Adjust the range to fetch more pages if needed
        start = page * rows_per_page
        # Define the URL with pagination
        url = f'https://mc-api.marketcheck.com/v2/search/car/active?api_key={api_key}&fuel_type={fuel_type}&car_type={car_type}&start={start}&rows={rows_per_page}'
        response = requests.get(url)
        data = response.json()
        listings = data.get('listings', [])

        temp_df = pd.DataFrame(listings)
        gas_df = pd.concat([gas_df, temp_df], ignore_index=True)

# Create a new df with the columns we want
get_gas_data(api_key, fuel_type, car_type, rows_per_page)
gas_df = gas_df[['heading', 'price', 'miles', 'first_seen_at_date']]
gas_df.head()

,heading,price,miles,first_seen_at_date
0,2018 LEXUS RX 350 SUV,30950.0,59908.0,2025-03-31T13:29:29.000Z
1,2015 Chevrolet Sonic LT Auto,9495.0,75541.0,2025-02-20T20:24:27.000Z
2,2021 LEXUS UX 250h SUV,31495.0,24499.0,2025-04-12T04:27:16.000Z
3,2024 Ford U0G Active,22597.0,22475.0,2025-04-13T03:05:02.000Z
4,2023 Ford Escape Active SUV,22475.0,23126.0,2025-04-12T03:23:19.000Z


In [2]:
import pandas as pd
import requests

ev_df = pd.DataFrame()
api_key = 'gxsUNM0ko2FvmZc77RZpoTsZTmONN3UP'
fuel_type = 'Electric'
car_type = 'used'
rows_per_page = 25

def get_ev_data(api_key, fuel_type, car_type, rows_per_page):
    global ev_df
    for page in range(0, 40):  # Adjust the range to fetch more pages if needed
        start = page * rows_per_page
        # Define the URL with pagination
        url = f'https://mc-api.marketcheck.com/v2/search/car/active?api_key={api_key}&fuel_type={fuel_type}&car_type={car_type}&start={start}&rows={rows_per_page}'
        response = requests.get(url)
        data = response.json()
        listings = data.get('listings', [])

        temp_df = pd.DataFrame(listings)
        ev_df = pd.concat([ev_df, temp_df], ignore_index=True)

# Create a new df with the columns we want
get_ev_data(api_key, fuel_type, car_type, rows_per_page)
ev_df = ev_df[['heading', 'price', 'miles', 'first_seen_at_date']]
ev_df.tail()

,heading,price,miles,first_seen_at_date
495,2023 Genesis Electrifi...,36998.0,15246.0,2025-04-12T02:00:44.000Z
496,2023 Hyundai IONIQ 6 Limited AWD Sedan,30663.0,12701.0,2025-04-10T02:20:25.000Z
497,2023 TESLA MODEL Y LONG RANGE,33995.0,29659.0,2025-03-13T05:11:38.000Z
498,2023 LEXUS RZ 450e Premium SUV,38989.0,4198.0,2025-01-22T12:25:53.000Z
499,Used 2022 Tesla Model Y,23721.0,23993.0,2025-03-06T03:31:28.000Z


In [3]:
# Change first_seen_at_date to datetime with only year-month
ev_df.sort_values(by='first_seen_at_date', ascending=True, inplace=True)
ev_df['first_seen_at_date'] = pd.to_datetime(ev_df['first_seen_at_date']).dt.strftime('%Y-%m')
ev_df.head()

,heading,price,miles,first_seen_at_date
435,Certified Pre-Owned 2020 Nissan Leaf SV,22375.0,23332.0,2023-08
136,Certified Pre-Owned 2021 Nissan Leaf SV,24425.0,10766.0,2023-08
478,Certified Pre-Owned 2020 Nissan Leaf SV,22050.0,19187.0,2023-08
41,2018 Tesla Model 3 Long Range,NaN,26160.0,2023-11
482,2017 Chevrolet Bolt EV,NaN,22416.0,2024-08


In [4]:
# Fix the date format and sort the DataFrame in chronological order
gas_df.sort_values(by='first_seen_at_date', ascending=True, inplace=True)
gas_df['first_seen_at_date'] = pd.to_datetime(gas_df['first_seen_at_date']).dt.strftime('%Y-%m')
gas_df.head()

,heading,price,miles,first_seen_at_date
135,2000 Chrysler LHS,NaN,133377.0,2023-06
77,2013 Chevrolet Sonic LTZ Auto,10995.0,131325.0,2023-08
51,2007 Chevrolet Cobalt,6995.0,100387.0,2023-08
173,2014 Ford F-150 Lariat,9950.0,199212.0,2023-09
125,2021 Hyundai Kona SEL Plus SUV,NaN,14630.0,2023-12


In [5]:

gas_df['first_seen_at_date'] = pd.to_datetime(gas_df['first_seen_at_date'], format='%Y-%m')

def clean_gas_df(df):
    # Define the start date and the interval
    start_date = pd.to_datetime('2023-08', format='%Y-%m')
    interval = pd.DateOffset(months=3)

    # Iterate through the DataFrame in 3-month intervals
    current_date = start_date
    means = []
    while current_date <= df['first_seen_at_date'].max():
        end_date = current_date + interval
        spliced_gas_df = df[(df['first_seen_at_date'] >= current_date) & (df['first_seen_at_date'] < end_date)]
        spliced_gas_df = spliced_gas_df[['miles','first_seen_at_date']]
        print(f"Data from {current_date.strftime('%Y-%m')} to {end_date.strftime('%Y-%m')}:")
        mu = spliced_gas_df['miles'].mean()
        print(mu)
        means.append(mu)
        current_date = end_date
    return means, spliced_gas_df

gas_means, spliced_gas_df = clean_gas_df(gas_df)

Data from 2023-08 to 2023-11:
143641.33333333334
Data from 2023-11 to 2024-02:
23541.666666666668
Data from 2024-02 to 2024-05:
119593.66666666667
Data from 2024-05 to 2024-08:
66920.0
Data from 2024-08 to 2024-11:
126877.30769230769
Data from 2024-11 to 2025-02:
106477.63888888889
Data from 2025-02 to 2025-05:
69768.69590643275


In [6]:

ev_df['first_seen_at_date'] = pd.to_datetime(ev_df['first_seen_at_date'], format='%Y-%m')

def clean_ev_df(df):
    # Define the start date and the interval
    start_date = pd.to_datetime('2023-08', format='%Y-%m')
    interval = pd.DateOffset(months=3)

    # Iterate through the DataFrame in 3-month intervals
    current_date = start_date
    means = []
    while current_date <= df['first_seen_at_date'].max():
        end_date = current_date + interval
        spliced_ev_df = df[(df['first_seen_at_date'] >= current_date) & (df['first_seen_at_date'] < end_date)]
        spliced_ev_df = spliced_ev_df[['miles','first_seen_at_date']]
        print(f"Data from {current_date.strftime('%Y-%m')} to {end_date.strftime('%Y-%m')}:")
        mu = spliced_ev_df['miles'].mean()
        print(f"Mean miles: {mu}")
        means.append(mu)
        current_date = end_date

    return means, spliced_ev_df

ev_means, spliced_ev_df = clean_ev_df(ev_df)

Data from 2023-08 to 2023-11:
Mean miles: 17761.666666666668
Data from 2023-11 to 2024-02:
Mean miles: 26160.0
Data from 2024-02 to 2024-05:
Mean miles: nan
Data from 2024-05 to 2024-08:
Mean miles: nan
Data from 2024-08 to 2024-11:
Mean miles: 14355.0
Data from 2024-11 to 2025-02:
Mean miles: 33204.818181818184
Data from 2025-02 to 2025-05:
Mean miles: 35649.705627705625


In [7]:
# Calculate ennergy displacement
displacements = []
consumptions = []
net_displacements = []
def calculate_energy_displacement_consumption(ev_means, gas_means):
    for ev_mean, gas_mean in zip(ev_means, gas_means):
        # Use estimated values for avg mpge and mpg
        displacement = (ev_mean * 0.34) / 33.7
        displacements.append(displacement)

        consumption = gas_mean/28
        consumptions.append(consumption)

        net_displacement = displacement - consumption
        net_displacements.append(net_displacement)
    return displacements, consumptions, net_displacements

displacements, consumptions, net_displacements = calculate_energy_displacement_consumption(ev_means, gas_means)

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Date': [f"{date.strftime('%Y-%m')}" for date in pd.date_range(start='2023-08-01', periods=len(displacements), freq='3MS')],
    'EV Mean Miles': ev_means,
    'Gas Mean Miles': gas_means,
    'Energy Displacement (GGE)': displacements,
    'Consumption (GGE)': consumptions,
    'Net Displacement (GGE)': net_displacements
})

results_df
    

,Date,EV Mean Miles,Gas Mean Miles,Energy Displacement (GGE),Consumption (GGE),Net Displacement (GGE)
0,2023-08,17761.666667,143641.333333,179.197824,5130.047619,-4950.849795
1,2023-11,26160.000000,23541.666667,263.928783,840.773810,-576.845026
2,2024-02,NaN,119593.666667,NaN,4271.202381,NaN
3,2024-05,NaN,66920.000000,NaN,2390.000000,NaN
4,2024-08,14355.000000,126877.307692,144.827893,4531.332418,-4386.504524
5,2024-11,33204.818182,106477.638889,335.004100,3802.772817,-3467.768717
6,2025-02,35649.705628,69768.695906,359.670621,2491.739140,-2132.068519


There's like somewhat of a trend with an increasing Net Displacement (meaning the difference between the ev usage/efficiency and gas is decreasing)

I just dk if the factor has enough data points for a time series or whatever model its gonna be inputted into. 